In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline
# Bước 1: Đọc dữ liệu
df = pd.read_csv('data/xAPI-Edu-Data.csv')
df = df.drop(columns=['StageID','GradeID','SectionID','Semester'])
print(df.isnull().sum())

gender                      0
NationalITy                 0
PlaceofBirth                0
Topic                       0
Relation                    0
raisedhands                 0
VisITedResources            0
AnnouncementsView           0
Discussion                  0
ParentAnsweringSurvey       0
ParentschoolSatisfaction    0
StudentAbsenceDays          0
Class                       0
dtype: int64


In [3]:
le = LabelEncoder()
df['gender'] = le.fit_transform(df['gender'])
df['NationalITy'] = le.fit_transform(df['NationalITy'])
df['PlaceofBirth'] = le.fit_transform(df['PlaceofBirth'])
df['Topic'] = le.fit_transform(df['Topic'])
df['Relation'] = le.fit_transform(df['Relation'])
df['ParentAnsweringSurvey'] = le.fit_transform(df['ParentAnsweringSurvey'])
df['ParentschoolSatisfaction'] = le.fit_transform(df['ParentschoolSatisfaction'])
df['StudentAbsenceDays'] = le.fit_transform(df['StudentAbsenceDays'])
df['Class'] = df['Class'].map({'L': 0, 'M': 1, 'H': 1})

# Chuẩn hóa các giá trị số
scaler = StandardScaler()
numeric_columns = ['raisedhands', 'VisITedResources', 'AnnouncementsView', 'Discussion']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

# In dữ liệu đã chuẩn hóa
print(df.head())


   gender  NationalITy  PlaceofBirth  Topic  Relation  raisedhands  \
0       1            4             4      7         0    -1.033429   
1       1            4             4      7         0    -0.870813   
2       1            4             4      7         0    -1.196046   
3       1            4             4      7         0    -0.545579   
4       1            4             4      7         0    -0.220346   

   VisITedResources  AnnouncementsView  Discussion  ParentAnsweringSurvey  \
0         -1.174075          -1.351167   -0.843326                      1   
1         -1.053029          -1.313549   -0.662225                      1   
2         -1.446426          -1.426401   -0.481125                      0   
3         -0.901723          -1.238315   -0.300024                      0   
4         -0.145191          -0.974994    0.243279                      0   

   ParentschoolSatisfaction  StudentAbsenceDays  Class  
0                         1                   1      1  
1 

In [4]:
# Bước 6: Tách X (features) và y (label)
X = df.drop('Class', axis=1)
y = df['Class']

# Chia train-test với stratify
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [15]:
# Grid tham số cho ANN + RFE
ann_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],
        'ann__hidden_layer_sizes': [(50,), (100,), (150,)],
        'ann__activation': ['identity', 'logistic'],
        'ann__learning_rate_init': [0.001, 0.01,0.1]
    }
]

# Pipeline ANN + RFE
pipeline = Pipeline([
    ('rfe', RFE(estimator=SVC(kernel= 'linear', random_state=42))),
    ('ann', MLPClassifier(random_state=42, max_iter=1000))
])

ann_results = []

# GridSearchCV
ann_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=ann_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

ann_grid.fit(X_train, y_train)

# Dự đoán test set
best_ann_pipeline = ann_grid.best_estimator_
y_pred_test = best_ann_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
ann_results.append({
    'best_params': ann_grid.best_params_,
    'validation_accuracy': ann_grid.best_score_,
    'test_accuracy': test_accuracy
})

# In kết quả
print(f"[ANN Pipeline] best_params={ann_grid.best_params_}")
print(f"[Validation Accuracy]: {ann_grid.best_score_:.4f}")
print(f"[Test Accuracy]: {test_accuracy:.4f}")

# Xuất ra CSV
pd.DataFrame(ann_results).to_csv('data2/ann_pipeline_results.csv', index=False)


[ANN Pipeline] best_params={'ann__activation': 'identity', 'ann__hidden_layer_sizes': (50,), 'ann__learning_rate_init': 0.01, 'rfe__n_features_to_select': 10}
[Validation Accuracy]: 0.9220
[Test Accuracy]: 0.9479


In [13]:
# Grid tham số cho ANN + RFE
ann_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],
        'ann__hidden_layer_sizes': [(50,), (100,), (150,)],
        'ann__activation': ['relu', 'tanh'],
        'ann__learning_rate_init': [0.001, 0.01]
    }
]

# Pipeline ANN + RFE
pipeline = Pipeline([
    ('rfe', RFE(estimator=DecisionTreeClassifier(random_state=42))),
    ('ann', MLPClassifier(random_state=42, max_iter=1000))
])

ann_results = []

# GridSearchCV
ann_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=ann_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

ann_grid.fit(X_train, y_train)

# Dự đoán test set
best_ann_pipeline = ann_grid.best_estimator_
y_pred_test = best_ann_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
ann_results.append({
    'best_params': ann_grid.best_params_,
    'validation_accuracy': ann_grid.best_score_,
    'test_accuracy': test_accuracy
})

# In kết quả
print(f"[ANN Pipeline] best_params={ann_grid.best_params_}")
print(f"[Validation Accuracy]: {ann_grid.best_score_:.4f}")
print(f"[Test Accuracy]: {test_accuracy:.4f}")

# Xuất ra CSV
pd.DataFrame(ann_results).to_csv('data2/ann1_pipeline_results.csv', index=False)


[ANN Pipeline] best_params={'ann__activation': 'tanh', 'ann__hidden_layer_sizes': (50,), 'ann__learning_rate_init': 0.001, 'rfe__n_features_to_select': 10}
[Validation Accuracy]: 0.9348
[Test Accuracy]: 0.9583


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [72]:
# Grid tham số cho SVM + RFE
svm_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],
        'svm__C': [0.5, 1, 0.1],
        'svm__kernel': ['sigmoid'],
        'svm__gamma': [1, 0.01, 0.1]
    }
]

# Pipeline SVM + RFE
pipeline = Pipeline([
    ('rfe', RFE(estimator=SVC(kernel='linear', random_state=42))),
    ('svm', SVC(random_state=42))
])

svm_results = []

# GridSearchCV
svm_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=svm_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

# Huấn luyện mô hình
svm_grid.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
best_svm_pipeline = svm_grid.best_estimator_
y_pred_test = best_svm_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
svm_results.append({
    'best_params': svm_grid.best_params_,
    'validation_accuracy': svm_grid.best_score_,
    'test_accuracy': test_accuracy
})

# In kết quả
print(f"[SVM Pipeline] best_params={svm_grid.best_params_}")
print(f"[Validation Accuracy]: {svm_grid.best_score_:.4f}")
print(f"[Test Accuracy]: {test_accuracy:.4f}")

# Xuất ra CSV
pd.DataFrame(svm_results).to_csv('data2/svm_pipeline_results.csv', index=False)


[SVM Pipeline] best_params={'rfe__n_features_to_select': 7, 'svm__C': 1, 'svm__gamma': 0.01, 'svm__kernel': 'sigmoid'}
[Validation Accuracy]: 0.8827
[Test Accuracy]: 0.9375


In [71]:
# Grid tham số cho SVM + RFE
svm_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],
        'svm__C': [0.01, 0.001],
        'svm__kernel': ['linear'],
        'svm__gamma': [0.1, 1, 10]
    }
]

# Pipeline SVM + RFE
pipeline = Pipeline([
    ('rfe', RFE(estimator=DecisionTreeClassifier(random_state=42))),
    ('svm', SVC(random_state=42))
])

svm_results = []

# GridSearchCV
svm_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=svm_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

# Huấn luyện mô hình
svm_grid.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
best_svm_pipeline = svm_grid.best_estimator_
y_pred_test = best_svm_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
svm_results.append({
    'best_params': svm_grid.best_params_,
    'validation_accuracy': svm_grid.best_score_,
    'test_accuracy': test_accuracy
})

# In kết quả
print(f"[SVM Pipeline] best_params={svm_grid.best_params_}")
print(f"[Validation Accuracy]: {svm_grid.best_score_:.4f}")
print(f"[Test Accuracy]: {test_accuracy:.4f}")

# Xuất ra CSV
pd.DataFrame(svm_results).to_csv('data2/svm1_pipeline_results.csv', index=False)


[SVM Pipeline] best_params={'rfe__n_features_to_select': 9, 'svm__C': 0.01, 'svm__gamma': 0.1, 'svm__kernel': 'linear'}
[Validation Accuracy]: 0.8827
[Test Accuracy]: 0.9375


In [23]:
# Grid tham số cho Decision Tree + RFE
dt_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],
        'dt__criterion': ['entropy'],
        'dt__max_depth': [3, 5],
        'dt__min_samples_split': [2, 4, 8]
    }
]

# Pipeline RFE + DecisionTree
pipeline = Pipeline([
    ('rfe', RFE(estimator=DecisionTreeClassifier(random_state=42))),
    ('dt', DecisionTreeClassifier(random_state=42))
])

dt_results = []

# GridSearchCV
dt_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=dt_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

# Huấn luyện mô hình
dt_grid.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
best_dt_pipeline = dt_grid.best_estimator_
y_pred_test = best_dt_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
dt_results.append({
    'best_params': dt_grid.best_params_,
    'validation_accuracy': dt_grid.best_score_,
    'test_accuracy': test_accuracy
})

# In kết quả
print(f"[DecisionTree Pipeline] best_params={dt_grid.best_params_}")
print(f"[Validation Accuracy]: {dt_grid.best_score_:.4f}")
print(f"[Test Accuracy]: {test_accuracy:.4f}")

# Xuất ra CSV
pd.DataFrame(dt_results).to_csv('data2/dt1_pipeline_results.csv', index=False)


[DecisionTree Pipeline] best_params={'dt__criterion': 'entropy', 'dt__max_depth': 5, 'dt__min_samples_split': 4, 'rfe__n_features_to_select': 9}
[Validation Accuracy]: 0.8932
[Test Accuracy]: 0.9167


In [20]:
# Grid tham số cho Decision Tree + RFE
dt_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],
        'dt__criterion': ['entropy'],
        'dt__max_depth': [3, 7, None],
        'dt__min_samples_split': [2, 4]
    }
]

# Pipeline RFE + DecisionTree
pipeline = Pipeline([
    ('rfe', RFE(estimator=SVC(kernel='linear', random_state=42))),
    ('dt', DecisionTreeClassifier(random_state=42))
])

dt_results = []

# GridSearchCV
dt_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=dt_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1
)

# Huấn luyện mô hình
dt_grid.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
best_dt_pipeline = dt_grid.best_estimator_
y_pred_test = best_dt_pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
dt_results.append({
    'best_params': dt_grid.best_params_,
    'validation_accuracy': dt_grid.best_score_,
    'test_accuracy': test_accuracy
})

# In kết quả
print(f"[DecisionTree Pipeline] best_params={dt_grid.best_params_}")
print(f"[Validation Accuracy]: {dt_grid.best_score_:.4f}")
print(f"[Test Accuracy]: {test_accuracy:.4f}")

# Xuất ra CSV
pd.DataFrame(dt_results).to_csv('data2/dt_pipeline_results.csv', index=False)


[DecisionTree Pipeline] best_params={'dt__criterion': 'entropy', 'dt__max_depth': None, 'dt__min_samples_split': 2, 'rfe__n_features_to_select': 8}
[Validation Accuracy]: 0.9011
[Test Accuracy]: 0.9167


In [18]:
# Danh sách các không gian tham số khác nhau mà bạn muốn thử
ada_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],  # Chọn số lượng đặc trưng trong RFE
        'adaboost__n_estimators': [150, 200, 250],
        'adaboost__estimator__criterion': ['entropy'],
        'adaboost__estimator__max_depth': [2, 3, 7],
        'adaboost__estimator__min_samples_split': [2, 6]
    }
]

# Lưu trữ tất cả kết quả và thông tin đặc trưng đã chọn
a_results = []

# Tạo pipeline
adaboost_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    random_state=42,
    algorithm='SAMME'
)

# Tạo pipeline với bước chọn đặc trưng RFE và mô hình AdaBoost
pipeline = Pipeline([
    ('rfe', RFE(estimator=DecisionTreeClassifier(random_state=42))),
    ('adaboost', adaboost_model)
])

# Lặp qua từng bộ tham số trong ada_param_grid
ada_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=ada_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1,  # Dùng hết CPU cho nhanh
)

# Huấn luyện mô hình và thu thập kết quả
ada_grid.fit(X_train, y_train)

# Đánh giá mô hình tốt nhất trên tập kiểm tra
best_model = ada_grid.best_estimator_
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
result = {
    'best_params': ada_grid.best_params_,
    'accuracy': test_accuracy
}
a_results.append(result)

# In ra kết quả sau khi huấn luyện xong
print(f"[Done Grid] random_state={42} | "
      f"test_accuracy={test_accuracy:.4f} | best_params={ada_grid.best_params_}")
print("===================")

# Xuất kết quả ra file CSV
a_results = pd.DataFrame(a_results)
a_results.to_csv('data2/adaboost1_results.csv', index=False)


[Done Grid] random_state=42 | test_accuracy=0.9792 | best_params={'adaboost__estimator__criterion': 'entropy', 'adaboost__estimator__max_depth': 7, 'adaboost__estimator__min_samples_split': 6, 'adaboost__n_estimators': 250, 'rfe__n_features_to_select': 8}


In [75]:
# Danh sách các không gian tham số khác nhau mà bạn muốn thử
ada_param_grid = [
    {
        'rfe__n_features_to_select': [7, 8, 9, 10],  # Chọn số lượng đặc trưng trong RFE
        'adaboost__n_estimators': [150, 200, 250],
        'adaboost__estimator__criterion': ['entropy'],
        'adaboost__estimator__max_depth': [None, 3, 5],
        'adaboost__estimator__min_samples_split': [2, 4, 8]
    }
]

# Lưu trữ tất cả kết quả và thông tin đặc trưng đã chọn
a_results = []

# Tạo pipeline
adaboost_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    random_state=42,
    algorithm='SAMME'
)

# Tạo pipeline với bước chọn đặc trưng RFE và mô hình AdaBoost
pipeline = Pipeline([
    ('rfe', RFE(estimator=SVC(kernel='linear', random_state=42))),
    ('adaboost', adaboost_model)
])

# Lặp qua từng bộ tham số trong ada_param_grid
ada_grid = GridSearchCV(
    estimator=pipeline,
    param_grid=ada_param_grid,
    cv=10,
    scoring='accuracy',
    n_jobs=-1,  # Dùng hết CPU cho nhanh
)

# Huấn luyện mô hình và thu thập kết quả
ada_grid.fit(X_train, y_train)

# Đánh giá mô hình tốt nhất trên tập kiểm tra
best_model = ada_grid.best_estimator_
y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Lưu kết quả
result = {
    'best_params': ada_grid.best_params_,
    'accuracy': test_accuracy
}
a_results.append(result)

# In ra kết quả sau khi huấn luyện xong
print(f"[Done Grid] random_state={42} | "
      f"test_accuracy={test_accuracy:.4f} | best_params={ada_grid.best_params_}")
print("===================")

# Xuất kết quả ra file CSV
a_results = pd.DataFrame(a_results)
a_results.to_csv('data2/adaboost_results.csv', index=False)


[Done Grid] random_state=42 | test_accuracy=0.9583 | best_params={'adaboost__estimator__criterion': 'entropy', 'adaboost__estimator__max_depth': 5, 'adaboost__estimator__min_samples_split': 2, 'adaboost__n_estimators': 200, 'rfe__n_features_to_select': 10}
